# 다층 퍼셉트론 기본 구조

## 전복 고리수 추정 신경망

In [1]:
!wget -O abalone.csv https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data

--2021-02-05 04:27:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [application/x-httpd-php]
Saving to: ‘abalone.csv’

abalone.csv         100%[===================>] 187.38K   416KB/s    in 0.5s    

2021-02-05 04:27:36 (416 KB/s) - ‘abalone.csv’ saved [191873/191873]



### 파이썬 모듈 불러들이기

In [2]:
import numpy as np
import csv
import time

np.random.seed(1234)

def randomize():
    np.random.seed(time.time())

### 하이퍼파라미터값의 정의

In [3]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

### 메인 함수 정의

In [4]:
def abalone_exec(epoch_count=10, mb_size=10, report=1): 
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

### 데이터 적재 함수 정의

In [5]:
def load_abalone_dataset():
    with open('abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        rows = []
        for row in csvreader:
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': 
            data[n, 0] = 1
        if row[0] == 'M': 
            data[n, 1] = 1
        if row[0] == 'F':
            data[n, 2] = 1
        data[n, 3:] = row[1:]       

### 은닉 계층 하나를 위한 파라미터 생성 함수 정의

In [6]:
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

### 학습 및 평가 함수 정의

In [7]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

### 학습 및 평가  데이터 획득 함수 정의

In [8]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

### 학습 실행 함수와 평가 실행 함수 정의

In [9]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

### 은닉 계층 하나를 위한 순전파 함수 정의

In [10]:
def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0)

### 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [11]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

### 은닉 계층 하나를 위한 역전파 함수 정의

In [12]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux

    g_output_w_out = hidden.transpose()                      
    G_w_out = np.matmul(g_output_w_out, G_output)            
    G_b_out = np.sum(G_output, axis=0)                       

    g_output_hidden = pm_output['w'].transpose()             
    G_hidden = np.matmul(G_output, g_output_hidden)          

    pm_output['w'] -= LEARNING_RATE * G_w_out                
    pm_output['b'] -= LEARNING_RATE * G_b_out                
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()                           
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)            
    G_b_hid = np.sum(G_hidden, axis=0)                       
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid                
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid                
    
def relu_derv(y):
    return np.sign(y)

### 가변적 은닉 계층 구성을 위한 파라미터 생성 함수 정의

In [13]:
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

### 가변적 은닉 계층 구성을 위한 순전파 함수 정의

In [14]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

### 가변적 은닉 계층 구성을 위한 역전파 함수 정의

In [15]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)

    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

### 스위치 함수 정의

In [16]:
global hidden_config

def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden1()
    
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)

### 은닉 계층 구조 지정 함수 정의

In [17]:
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info

### 정확도 계산 함수 정의

In [18]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

## 실행하기

### 단층 퍼셉트론 처리하기

In [19]:
set_hidden([])
abalone_exec()

은닉 계층 0개를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810


### 은닉 계층 하나로 풀어보는 전복 고리 수 추정 문제

In [20]:
set_hidden(4)
abalone_exec(epoch_count=50, report=10)

은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 10: loss=6.849, accuracy=0.806/0.803
Epoch 20: loss=6.294, accuracy=0.815/0.812
Epoch 30: loss=5.572, accuracy=0.827/0.840
Epoch 40: loss=5.105, accuracy=0.835/0.842
Epoch 50: loss=4.955, accuracy=0.838/0.837

Final Test: final accuracy = 0.837


### 난수 함수의 효과 확인

In [21]:
abalone_exec(epoch_count=50, report=10)

은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.
Epoch 10: loss=6.757, accuracy=0.808/0.813
Epoch 20: loss=6.255, accuracy=0.817/0.817
Epoch 30: loss=5.554, accuracy=0.829/0.819
Epoch 40: loss=5.093, accuracy=0.837/0.845
Epoch 50: loss=4.948, accuracy=0.840/0.827

Final Test: final accuracy = 0.827
